# Lesson: Manipulating Atoms in Python3

In [1]:
import sys
sys.path

['',
 '/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '/home/tensae/.local/lib/python3.5/site-packages',
 '/usr/local/lib/python3.5/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/IPython/extensions',
 '/home/tensae/.ipython']

### Making some Atoms (python)

Here I will mostly use the ipython shell to give examples. But you can of course put all the commands in a script if you like. Import AtomSpace and types to start making atoms.

In [2]:
from opencog.atomspace import AtomSpace, types

There are two ways to create Atoms in python. You can use the c++ like syntax or you can use the scheme like syntax.

__c++ like syntax__

In [3]:
atsp = AtomSpace()

In [4]:
cat = atsp.add_node(types.ConceptNode, "Cat")

In [5]:
animal = atsp.add_node(types.ConceptNode, "Animal")

In [6]:
man = atsp.add_node(types.ConceptNode, "Man")

In [7]:
man

(ConceptNode "Man") ; [4943495822049051872][1]

So printing the AtomSpace didn't quite work out as we might have expected from c++. We can print its contents as follows:

In [8]:
for atom in atsp:
    print (atom)

(ConceptNode "Animal")

(ConceptNode "Man")

(ConceptNode "Cat")



Now lets add some links into the AtomSpace. This is again, just like c++.

In [9]:
atsp.add_link(types.InheritanceLink, [man, animal])

(InheritanceLink
  (ConceptNode "Man") ; [4943495822049051872][1]
  (ConceptNode "Animal") ; [1051245045201563895][1]
) ; [17225707082406533815][1]

__scheme like syntax__

To use scheme like concise syntax you have to do a few extra steps. Import the type_constructors that enable you to add atoms to the AtomSpace by calling functions named after their types. Import initialize_opencog a function that binds a AtomSpace instance to the type_constructors.

In [10]:
from opencog.utilities import initialize_opencog

In [11]:
from opencog.type_constructors import *

Now you can add atoms just like in scheme. This becomes very useful when you start building complex patterns for the pattern matcher.

In [12]:
initialize_opencog(atsp)

In [13]:
color = ConceptNode("Color")

In [14]:
c1 = ConceptNode("Blue")

In [15]:
link = InheritanceLink(c1, color)

In [16]:
for atom in atsp:
    print (atom)

(ConceptNode "Blue")

(ConceptNode "Cat")

(ConceptNode "Man")

(ConceptNode "Animal")

(ConceptNode "Color")

(InheritanceLink
  (ConceptNode "Blue")
  (ConceptNode "Color")
)

(InheritanceLink
  (ConceptNode "Man")
  (ConceptNode "Animal")
)



One last thing before we wrap this up. The Atoms are classes with a lot of useful methods. If you are in the ipython shell you can look at a list by writing <variable_pointing_to_an_atom>.<Tab>. For example we have a variable color that points to the ConceptNode object of name "Color". Then I can see a list of all the method I can call on it by pressing tab to view the options.

In [17]:
color.t

18

#### Pattern Matching

Now lets look at how we can run the PM through python. The PM functions are defined in c++ and made available in python through wrappers that are defined in atomspace/opencog/cython/opencog/bindlink.pyx and you can see the imports in atomspace/opencog/cython/opencog/bindlink.pxd

Here we will again run the equivalent of bindlink and cog-staisfying-set functions in scheme.

In [18]:
#Import the staisfying_set function
from opencog.bindlink import satisfying_set

In [19]:
#Delete existing atoms
atsp.clear()

In [20]:
#Create some atoms
InheritanceLink(ConceptNode("Red"), color)

(InheritanceLink
  (ConceptNode "Red") ; [6709592507294448610][1]
  (ConceptNode "Color") ; [8429248309562626666][1]
) ; [18321296701882258284][1]

In [21]:
InheritanceLink(ConceptNode("Green"), color)

(InheritanceLink
  (ConceptNode "Green") ; [7362812596333203375][1]
  (ConceptNode "Color") ; [8429248309562626666][1]
) ; [12207443529596838105][1]

In [22]:
InheritanceLink(ConceptNode("Blue"), color)

(InheritanceLink
  (ConceptNode "Blue") ; [3109877522812845760][1]
  (ConceptNode "Color") ; [8429248309562626666][1]
) ; [10211166656246673930][1]

In [23]:
#Create a pattern to look for color nodes
varlink = TypedVariableLink(VariableNode("$xcol"), TypeNode("ConceptNode"))

In [24]:
pattern = InheritanceLink(VariableNode("$xcol"), color)

In [25]:
colornodes = GetLink(varlink, pattern)

In [26]:
satisfying_set(atsp, colornodes)

(SetLink
  (ConceptNode "Blue") ; [3109877522812845760][1]
  (ConceptNode "Red") ; [6709592507294448610][1]
  (ConceptNode "Green") ; [7362812596333203375][1]
) ; [17576040149376550613][1]

In [27]:
def something():
   
   """run the function to find all color nodes.
     
        >>> satisfying_set(atsp, colornodes)
        (SetLink
          (ConceptNode "Blue") ; [3109877522812845760][1]
          (ConceptNode "Red") ; [6709592507294448610][1]
          (ConceptNode "Green") ; [7362812596333203375][1]
        ) ; [17576040149376550613][1]
                                        
   """

In a similar manner you can import bindlink from opencog.bindlink and use it to execute rewrite queries (with the help of BindLink in place of SatisfactionLink)

In [28]:
import doctest

In [29]:
doctest.testmod(verbose=True)

Trying:
    satisfying_set(atsp, colornodes)
Expecting:
    (SetLink
      (ConceptNode "Blue") ; [3109877522812845760][1]
      (ConceptNode "Red") ; [6709592507294448610][1]
      (ConceptNode "Green") ; [7362812596333203375][1]
    ) ; [17576040149376550613][1]
**********************************************************************
Line 3, in __main__.something
Failed example:
    satisfying_set(atsp, colornodes)
Expected:
    (SetLink
      (ConceptNode "Blue") ; [3109877522812845760][1]
      (ConceptNode "Red") ; [6709592507294448610][1]
      (ConceptNode "Green") ; [7362812596333203375][1]
    ) ; [17576040149376550613][1]
Got:
    (SetLink
      (ConceptNode "Blue") ; [3109877522812845760][1]
      (ConceptNode "Red") ; [6709592507294448610][1]
      (ConceptNode "Green") ; [7362812596333203375][1]
    ) ; [17576040149376550613][1]
    <BLANKLINE>
1 items had no tests:
    __main__
**********************************************************************
1 items had failures:
   1

TestResults(failed=1, attempted=1)